## [Previous Section](./ClassesDecorator.ipynb)

## Enhancing Classes with a Class Decorator

In [2]:
from collections.abc import Callable  
from typing import Any, NoReturn, get_type_hints

class Field:
    def __init__(self, name: str, constructor: Callable) -> None:
        if not callable(constructor) or constructor is type(None):
            raise TypeError(f'{name!r} type hint must be callable')
        self.name = name
        self.constructor = constructor

    def __set__(self, instance: Any, value: Any) -> None:
        if value is ...:  # <4>
            value = self.constructor()
        else:
            try:
                value = self.constructor(value) 
            except (TypeError, ValueError) as e:
                type_name = self.constructor.__name__
                msg = (
                    f'{value!r} is not compatible with {self.name}:{type_name}'
                )
                raise TypeError(msg) from e
        instance.__dict__[self.name] = value

Every top-level function in this file is prefixed with an underscore, **except** the `checked` decorator. 

The methods to be injected in the decorated class. 

In [3]:
def _fields(cls: type) -> dict[str, type]:
    return get_type_hints(cls)

def __init__(self: Any, **kwargs: Any) -> None:
    for name in self._fields():
        value = kwargs.pop(name, ...)
        setattr(self, name, value)
    if kwargs:
        self.__flag_unknown_attrs(*kwargs)

def __setattr__(self: Any, name: str, value: Any) -> None:
    if name in self._fields():
        cls = self.__class__
        descriptor = getattr(cls, name)
        descriptor.__set__(self, value)
    else:
        self.__flag_unknown_attrs(name)

def __flag_unknown_attrs(self: Any, *names: str) -> NoReturn:
    plural = 's' if len(names) > 1 else ''
    extra = ', '.join(f'{name!r}' for name in names)
    cls_name = repr(self.__class__.__name__)
    raise AttributeError(f'{cls_name} has no attribute{plural} {extra}')

def _asdict(self: Any) -> dict[str, Any]:
    return {
        name: getattr(self, name)
        for name, attr in self.__class__.__dict__.items()
        if isinstance(attr, Field)
    }

def __repr__(self: Any) -> str:
    kwargs = ', '.join(
        f'{key}={value!r}' for key, value in self._asdict().items()
    )
    return f'{self.__class__.__name__}({kwargs})'




# Recall that classes are instances of type. 
# These type hints strongly suggest this is a class decorator: 
# it takes a class and returns a class. 
def checked(cls: type) -> type:  
    for name, constructor in _fields(cls).items():    
        setattr(cls, name, Field(name, constructor)) 

    cls._fields = classmethod(_fields)  # type: ignore  

    # Module-level functions that will become instance methods of the decorated class. 
    instance_methods = (  
        __init__,
        __repr__,
        __setattr__,
        _asdict,
        __flag_unknown_attrs,
    )
    # Add each of the `instance_methods` to `cls`. 
    for method in instance_methods: 
        setattr(cls, method.__name__, method)

    # Return the decorated `cls`, 
    # fulfilling the essential contract of a class decorator. 
    return cls

In [4]:
@checked
class Movie:
    title: str
    year: int
    box_office: float

In [15]:
movie = Movie(title='The Godfather', year=1972, box_office=137)
movie.title

'The Godfather'

In [6]:
movie

Movie(title='The Godfather', year=1972, box_office=137.0)

## Metaclasses 101

The bound method object also has a `__call__` method, which handles the actual invocation. This method calls the original function referenced in `__func__`, passing the `__self__` attribute of the method as the first argument. 

In [7]:
class LineItem:
    pass


print(
    str.__class__,
    LineItem.__class__,
    type.__class__)

<class 'type'> <class 'type'> <class 'type'>


They all are subclasees of `object`. 

In [8]:
from abc import ABCMeta
from collections.abc import Iterable

print(
    Iterable.__class__,
    ABCMeta.__class__)

<class 'abc.ABCMeta'> <class 'type'>


Note that `Iterable` is an abstract class, but `ABCMeta` is a concrete class. 

the class of `ABCMeta` is also `type`. Every class is an instance of `type`, directly or indirectly. 

`Iterable` is a subclass of `object` and an instance of `ABCMeta`. Both `object` and `ABCMeta` are instance of `type`, but the  key relationship here is that `ABCMeta` is also a subclass of `type`, because `ABCMeta` is a metaclass. 

### Customizes a Class

To use a metaclass, it's critical to understand how `__new__` works on any class. 

Consider this declaration:

```python
class Klass(SuperKlass, metaclass=MetaClass):
    x = 42
    def __init__(self, y):
        self.y = y
```

To process that `class` statement, Python calls `MetaClass.__new__` with these arguments:
- **`meta_cls`:** The metaclass itself(`MetaClass`), because `MetaClass.__new__` works as class method. 
- **`cls_name`:** The string `Klass`. 
- **`bases`:** The single-element tuple `(SuperKlass,)`, with more elements in the case of multiple inheritance. 
- **`cls_dict`:** A mapping like: `{x: 42, __init__:<function __init__ at 0x1009c4040>}`. 



When you implement `MetaClass.__new__`, you can inspect and change those arguments before passing them to `super().__new__`, which will eventually call `type.__new__` to create the new class object. 

After `super().__new__` returns, you can also apply further processing to the newly created class before returning it to Python. Python then calls `SuperClass.__init_subclass__`, passing the class you created, and then applies a class decorator to it, if one is present. Finally, Python binds the class object to its name in the surrounding namespace —— usually the gloabal namespace of a module, if the `class` statement was a top-level statement. 

The most common processiong made in a metaclass `__new__` is to add or replace items in the `cls_dict` —— the mapping that represents the namespace of the class under construction. For instance, before calling `super().__new__`, you can inject methods in the class under construction by adding functions to `cls_dict`. However, note that adding methods can also be done after the class is built, which is why we were able to do it using `__init_subclass__` or a class decorator. 

#### **A Metaclass Example**

In [9]:
# To create a new metaclass, inherit from `type`
class MetaBunch(type):

    # `__new__` works as a class method. 
    def __new__(meta_cls, cls_name, bases, cls_dict):
        # `defaults` will hold a mapping of attribute names and their default value. 
        defaults = {}

        # This will be injected into the new class. 
        def __init__(self, **kwargs):
            # Read from `defaults` and set the corresponding instance attribute with a value popped from `kwargs` or a default. 
            for name, default in defaults.items():
                setattr(self, name, kwargs.pop(name, default))
            # If there is still any item in `kwargs`, it means there are no slots left where we can place them. 
            # But we don't want to silently ignore extra items. A quick and effective solution is pop one item 
            # from `kwargs` and try to set it on the instance, triggering an AttributeError on purpose. 
            if kwargs:
                extra = ', '.join(kwargs)
                raise AttributeError(f'No slots left for: {extra!r}')
        
        def __repr__(self) -> str:
            rep = ', '.join(f'{name}={value!r}'
                        for name, default in defaults.items()
                        if (value := getattr(self, name)) != default)
            return f'{cls_name}({rep})'
    
        # Initialize namespace for the new class. 
        new_dict = dict(__slots__=[], __init__=__init__, __repr__=__repr__)

        for name, value in cls_dict.items():
            # If a dunder `name` is found, copyt the item to the new class namespace, 
            # unless it's already there. This prevents uses from overwriting `__init__`, `__repr__`
            # and other attributes set by Python, such as `__qualname__` and `__module__`. 
            if name.startswith('__') and name.endswith('__'):
                if name in new_dict:
                    raise AttributeError(f"Cant't set {name!r} in {cls_name!r}")
                new_dict[name] = value
            # If not a dunder `name`, append to `__slots__` and save its `value` in `defaults`. 
            else:
                new_dict['__slots__'].append(name)
                defaults[name] = value

        # Build and return the new class. 
        return super().__new__(meta_cls, cls_name, bases, new_dict)

`MetaBunch` works because it is able to configure `__slots__` before calling `super().__new__` to build the final class. 

Then, we provide a base class, so users don't need to see `MetaBunch`. 

In [10]:
class Bunch(metaclass=MetaBunch):
    pass

In [11]:
class Point(Bunch):
    x = 0.0
    y = 0.0
    color = 'gray'

In [12]:
Point(x=1.2, y=3, color='green')

Point(x=1.2, y=3, color='green')

In [13]:
p = Point()
p.x, p.y, p.color

(0.0, 0.0, 'gray')

In [14]:
p

Point()